In [5]:
import pandas as pd

df = pd.read_parquet('data/ada_002_embeddings.parquet')
df.head()

,document,chapter,section,link,contents,text,n_tokens,embeddings
0,Google C++ Style Guide,Background,,https://google.github.io/styleguide/cppguide.h...,C++ is one of the main development languages ...,Styleguide: Google C++ Style Guide - Backgroun...,232,"[0.027113765478134155, 0.03867639601230621, 0...."
1,Google C++ Style Guide,Background,Goals of the Style Guide,https://google.github.io/styleguide/cppguide.h...,Why do we have this document? There are a few...,Styleguide: Google C++ Style Guide - Backgroun...,1213,"[0.006240654271095991, 0.02487936243414879, 0...."
2,Google C++ Style Guide,C++ Version,,https://google.github.io/styleguide/cppguide.h...,"Currently, code should target C++20, i.e., sho...",Styleguide: Google C++ Style Guide - C++ Versi...,112,"[-0.00606873445212841, 0.016746634617447853, 0..."
3,Google C++ Style Guide,Header Files,,https://google.github.io/styleguide/cppguide.h...,"In general, every .cc file should have an asso...",Styleguide: Google C++ Style Guide - Header Fi...,116,"[-0.019454028457403183, 0.03280024603009224, -..."
4,Google C++ Style Guide,Header Files,Self-contained Headers,https://google.github.io/styleguide/cppguide.h...,Header files should be self-contained (compile...,Styleguide: Google C++ Style Guide - Header Fi...,318,"[-0.023608596995472908, 0.040658511221408844, ..."


In [7]:
!pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 6.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.5 MB/s eta 0:00:00


In [2]:
from sys import getsizeof

too_big = []

for text in df['text'].tolist():
    if getsizeof(text) > 5000:
        too_big.append((text, getsizeof(text)))

print(f"{len(too_big)} / {len(df)} records are too big")

6 / 106 records are too big


In [13]:
df['id'] = [str(i) for i in range(len(df))]
df.head()

,document,chapter,section,link,contents,text,n_tokens,embeddings,id
0,Google C++ Style Guide,Background,,https://google.github.io/styleguide/cppguide.h...,C++ is one of the main development languages ...,Styleguide: Google C++ Style Guide - Backgroun...,232,"[0.027113765478134155, 0.03867639601230621, 0....",0
1,Google C++ Style Guide,Background,Goals of the Style Guide,https://google.github.io/styleguide/cppguide.h...,Why do we have this document? There are a few...,Styleguide: Google C++ Style Guide - Backgroun...,1213,"[0.006240654271095991, 0.02487936243414879, 0....",1
2,Google C++ Style Guide,C++ Version,,https://google.github.io/styleguide/cppguide.h...,"Currently, code should target C++20, i.e., sho...",Styleguide: Google C++ Style Guide - C++ Versi...,112,"[-0.00606873445212841, 0.016746634617447853, 0...",2
3,Google C++ Style Guide,Header Files,,https://google.github.io/styleguide/cppguide.h...,"In general, every .cc file should have an asso...",Styleguide: Google C++ Style Guide - Header Fi...,116,"[-0.019454028457403183, 0.03280024603009224, -...",3
4,Google C++ Style Guide,Header Files,Self-contained Headers,https://google.github.io/styleguide/cppguide.h...,Header files should be self-contained (compile...,Styleguide: Google C++ Style Guide - Header Fi...,318,"[-0.023608596995472908, 0.040658511221408844, ...",4


In [9]:
import pinecone
import dotenv
import os

dotenv.load_dotenv('.env', override=True)

pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],  # app.pinecone.io
    environment=os.environ["PINECONE_ENV"]  # find next to API key in console
)

index_name = 'codestyle-semantic-search'

if not index_name in pinecone.list_indexes():
    pinecone.create_index(
        index_name, dimension=len(df['embeddings'].tolist()[0]),
        metric='cosine'
    )

index = pinecone.Index(index_name)

In [15]:
from tqdm.auto import tqdm

batch_size = 32

for i in tqdm(range(0, len(df), batch_size)):
    i_end = min(i+batch_size, len(df))
    df_slice = df.iloc[i:i_end]
    to_upsert = [
        (
            row['id'],
            row['embeddings'].tolist(),
            {
                'document': row['document'],
                'chapter': row['chapter'],
                'section': row['section'],
                'link': row['link'],
                'n_tokens': row['n_tokens']
            }
        ) for _, row in df_slice.iterrows()
    ]
    index.upsert(vectors=to_upsert)

100%|██████████| 4/4 [00:02<00:00,  1.37it/s]


In [16]:
mappings = {row['id']: row['text'] for _, row in df[['id', 'text']].iterrows()}

In [17]:
import json

with open('data/mapping.json', 'w') as fp:
    json.dump(mappings, fp)